In [1]:
from numpy.polynomial import Polynomial as P
import pandas as pd
import numpy as np
import math

In [5]:
data=pd.read_csv('./lightgbm_hyperpara_data_tuned.csv')

In [6]:
data

,Unnamed: 0,SN,number.leaves,Learning.Rate,number.iteration,maximum.bin,maximum.depth,acu,precision,recall,fscore,corr,training.error,test.error
0,1,0,0,0.000000,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,1,601,0.069850,10,3,4,0.831539,0.794733,0.272401,0.402513,0.552800,0.151236,0.197109
2,3,2,289,0.069850,10,3,4,0.828121,0.860706,0.226434,0.351436,0.551022,0.153860,0.201650
3,4,3,139,0.069850,10,3,4,0.833687,0.830441,0.266340,0.395179,0.558661,0.146739,0.197146
4,5,4,65,0.069850,10,3,4,0.823631,0.867133,0.296177,0.426909,0.542573,0.151055,0.189672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,117,116,601,0.058214,79,4,5,0.845330,0.609237,0.602564,0.605097,0.592645,0.039168,0.191921
117,118,117,289,0.058214,79,4,5,0.843447,0.582743,0.605361,0.592240,0.599089,0.041229,0.203888
118,119,118,139,0.058214,79,4,5,0.841213,0.580268,0.559021,0.566048,0.594859,0.038419,0.207655
119,120,119,65,0.058214,79,4,5,0.836916,0.547795,0.569277,0.551824,0.577744,0.039917,0.218145


In [7]:
num_leaves=list(data['number.leaves'])
Learning_rate=list(data['Learning.Rate'])
num_iter=list(data['number.iteration'])
max_bin=list(data['maximum.bin'])
max_depth=list(data['maximum.depth'])

In [8]:
Xlistlabel=['number.leaves','Learning.Rate','number.iteration','maximum.bin','maximum.depth']

In [9]:
Y=list(data['recall'])

In [10]:
def polyfunction(xx,yy,n):
        val=list(np.polyfit(xx,yy,n))
        return(val)

In [11]:
def fit(xx,yy,v):
    degree=len(v)-1
    predictedy=[]
    for i in range(len(xx)):
        temp=0.0
        for j in range(len(v)):
            temp+=float(v[j]*(xx[i]**(len(v)-j-1)))
        predictedy.append(temp)
    predarr=np.asarray(predictedy)
    yyarr=np.asarray(yy)
    if(len(predarr)==len(yyarr)):
        rmse=math.sqrt(sum((predarr-yyarr)**2)/len(yyarr))
    else:
        raise Valerror('true Y and predicted Y differes in dim')
    return(rmse)

In [12]:
def iterator(xx,yy):
    error=[]
    model=[]
    for i in range(10):
        eqa=polyfunction(xx,yy,i)
        errorr=fit(xx,yy,eqa)
        model.append(eqa)
        error.append(errorr)
    optimumdegree=error.index(min(error))
    return(optimumdegree,model[optimumdegree])

In [13]:
def partialdiff(x):
    degree,eqa=x
    lenx=len(eqa)
    for i in range(len(eqa)):
        eqa[i]=eqa[i]*(lenx-1-i)
    diffeqa=eqa
    return(diffeqa[:(lenx-1)])

In [15]:
response={}
for i in range(len(Xlistlabel)):
    response[Xlistlabel[i]]=iterator(list(data[Xlistlabel[i]]),Y)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: RankWarning: Polyfit may be poorly conditioned
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: RankWarning: Polyfit may be poorly conditioned
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: RankWarning: Polyfit may be poorly conditioned
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: RankWarning: Polyfit may be poorly conditioned
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: RankWarning: Polyfit may be poorly conditioned
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: RankWarning: Polyfit may be poorly conditioned
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/

In [16]:
roots={}
for keys in response:
    roots[keys]=np.roots(partialdiff(response[keys]))
    roots[keys]=roots[keys].real+roots[keys].imag

In [17]:
roots

{'number.leaves': array([513.51218715, 235.06291207, 106.55037639,  45.09441429]),
 'Learning.Rate': array([0.11104572]),
 'number.iteration': array([-82.55294878,  56.55185959,  -1.10182197]),
 'maximum.bin': array([ 3.47401269,  3.00265565, -3.26736639, -1.16921887, -4.04818863]),
 'maximum.depth': array([4.38020071])}

In [18]:
postiveroots={}
for keys in roots:
    postiveroots[keys]=roots[keys][roots[keys]>0]

In [19]:
postiveroots

{'number.leaves': array([513.51218715, 235.06291207, 106.55037639,  45.09441429]),
 'Learning.Rate': array([0.11104572]),
 'number.iteration': array([56.55185959]),
 'maximum.bin': array([3.47401269, 3.00265565]),
 'maximum.depth': array([4.38020071])}